<a href="https://colab.research.google.com/github/quinn-dougherty/DS-Unit-1-Sprint-4-Statistical-Tests-and-Experiments/blob/master/module1-statistics-probability-and-inference/LS_DS_141_Statistics_Probability_and_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science Module 141
## Statistics, Probability, and Inference

## Prepare - examine what's available in SciPy

As we delve into statistics, we'll be using more libraries - in particular the [stats package from SciPy](https://docs.scipy.org/doc/scipy/reference/tutorial/stats.html).

In [1]:
!pip install update pandas
import pandas as pd

from scipy import stats
# dir(stats)

pd.__version__

'0.22.0'

In [0]:
# As usual, lots of stuff here! There's our friend, the normal distribution
norm = stats.norm()
print(norm.mean())
print(norm.std())
print(norm.var())

0.0
1.0
1.0


In [0]:
# And a new friend - t
t1 = stats.t(5)  # 5 is df "shape" parameter
print(t1.mean())
print(t1.std())
print(t1.var())

0.0
1.2909944487358056
1.6666666666666667


![T distribution PDF with different shape parameters](https://upload.wikimedia.org/wikipedia/commons/4/41/Student_t_pdf.svg)

*(Picture from [Wikipedia](https://en.wikipedia.org/wiki/Student's_t-distribution#/media/File:Student_t_pdf.svg))*

The t-distribution is "normal-ish" - the larger the parameter (which reflects its degrees of freedom - more input data/features will increase it), the closer to true normal.

In [0]:
t2 = stats.t(30)  # Will be closer to normal
print(t2.mean())
print(t2.std())
print(t2.var())

0.0
1.0350983390135313
1.0714285714285714


Why is it different from normal? To better reflect the tendencies of small data and situations with unknown population standard deviation. In other words, the normal distribution is still the nice pure ideal in the limit (thanks to the central limit theorem), but the t-distribution is much more useful in many real-world situations.

History sidenote - this is "Student":

![William Sealy Gosset](https://upload.wikimedia.org/wikipedia/commons/4/42/William_Sealy_Gosset.jpg)

*(Picture from [Wikipedia](https://en.wikipedia.org/wiki/File:William_Sealy_Gosset.jpg))*

His real name is William Sealy Gosset, and he published under the pen name "Student" because he was not an academic. He was a brewer, working at Guinness and using trial and error to determine the best ways to yield barley. He's also proof that, even 100 years ago, you don't need official credentials to do real data science!

## Live Lecture - let's perform and interpret a t-test

We'll generate our own data, so we can know and alter the "ground truth" that the t-test should find. We will learn about p-values and how to interpret "statistical significance" based on the output of a hypothesis test.

In [0]:
# TODO - during class, but please help!

## Assignment - apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [160]:
from scipy import stats
import pandas as pd

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data"

'''
7. Attribute Information:
   1. Class Name: 2 (democrat, republican)
   2. handicapped-infants: 2 (y,n)
   3. water-project-cost-sharing: 2 (y,n)
   4. adoption-of-the-budget-resolution: 2 (y,n)
   5. physician-fee-freeze: 2 (y,n)
   6. el-salvador-aid: 2 (y,n)
   7. religious-groups-in-schools: 2 (y,n)
   8. anti-satellite-test-ban: 2 (y,n)
   9. aid-to-nicaraguan-contras: 2 (y,n)
  10. mx-missile: 2 (y,n)
  11. immigration: 2 (y,n)
  12. synfuels-corporation-cutback: 2 (y,n)
  13. education-spending: 2 (y,n)
  14. superfund-right-to-sue: 2 (y,n)
  15. crime: 2 (y,n)
  16. duty-free-exports: 2 (y,n)
  17. export-administration-act-south-africa: 2 (y,n)
'''

feature_names = ['Party_Affil', 
           'handicapped_infants', 
           'water_project_cost_sharing', 
           'adoption_of_the_budget_resolution', 
           'physician_fee_freeze',
           'el_salvador_aid',
           'religious_groups_in_schools',
           'anti_satellite_test_ban',
           'aid_to_nicaraguan_contras',
           'mx_missile',
           'immigration',
           'synfuels_corporation_cutback',
           'education_spending',
           'superfund_right_to_sue',
           'crime',
           'duty_free_exports',
           'export_administration_act_south_africa']

df_0 = pd.read_csv(url, header=None, names=feature_names)

df = df_0.copy()
numerize = {'y': 1, '?': 0.5, 'n': 0}

for feat in df.columns.drop('Party_Affil'):
  df[feat] = df_0[feat].map(numerize)

df.head()

,Party_Affil,handicapped_infants,water_project_cost_sharing,adoption_of_the_budget_resolution,physician_fee_freeze,el_salvador_aid,religious_groups_in_schools,anti_satellite_test_ban,aid_to_nicaraguan_contras,mx_missile,immigration,synfuels_corporation_cutback,education_spending,superfund_right_to_sue,crime,duty_free_exports,export_administration_act_south_africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.5,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.5
2,democrat,0.5,1.0,1.0,0.5,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,0.5,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.5,1.0,1.0,1.0,1.0


In [161]:
# Consider the inference "all republicans are worried about crime"

def repubs_yes(feat): 
  # 1-sample T-test whether republicans are concerned with a given issue. 
  return stats.ttest_1samp(df[feat][df.Party_Affil == 'republican'], 1)

repubs_yes('crime')

Ttest_1sampResult(statistic=-3.0555015916939365, pvalue=0.002616301602863148)

In [0]:

# Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
# Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01

def rep_more_than_dem(feat): 
  a = df[feat][df.Party_Affil == 'republican']
  b = df[feat][df.Party_Affil == 'democrat']
  rtn = stats.ttest_ind(a,b)
  return rtn

def dem_more_than_rep(feat): 
  a = df[feat][df.Party_Affil == 'republican']
  b = df[feat][df.Party_Affil == 'democrat']
  rtn = stats.ttest_ind(b,a)
  return rtn


In [0]:

## theorem 
for issue in df.columns[1:]:
  t_1 = rep_more_than_dem(issue)[0]
  t_2 = dem_more_than_rep(issue)[0]
  
  assert t_1 == (0 - 1) * t_2
  
  p_1 = rep_more_than_dem(issue)[1]
  p_2 = dem_more_than_rep(issue)[1]
  
  assert p_1==p_2
# in plain english: searching for "issue that republicans support more than 
# " is equivalent to searching for "issue that democrats search more than 
# republicans."

In [0]:
results = pd.DataFrame(columns=['T_stat', 'p_value'])
for issue in df.columns[1:]:
  results.loc[issue] = list(rep_more_than_dem(issue))

def min_with_index(rslts): 
  idx = rslts.p_value.idxmin()
  mina = rslts.p_value.min()
  return (idx, mina)

# What we really want is significant results. 
# so we'll delete the ones that fail p<0.01
significant_results = results[results.p_value < 0.01]

# also have to update dataframe
# df.drop(df[results.])

df.drop(list(results[results.p_value > 0.01].index), axis=1, inplace=True)

In [0]:
first_min = min_with_index(significant_results)

second_min = min_with_index(significant_results.drop(first_min[0], axis=0))

assert first_min[1] < 0.01
assert first_min[1] != second_min[1]
assert second_min[1] < 0.01


In [166]:
from numpy import sign
from decimal import Decimal
# if (result.loc[first_min[0]][1] < ):
#   result.loc[first_min[0]][1]

def print_report(issue):
  ## Since the assignment asked for just D or R but didn't care about magnitude, 
  # # # i'm gonna compress the t-stat into just sign. 
  sg = sign(significant_results.loc[issue][0])
  if (sg==1): 
    party = "Republican"
  else: 
    party = "Democrat"
  pv = "%.2E" % Decimal(results.loc[issue][1])
  s = "On the issue of {0}, support leans {1} at p={2}.\n".format(issue, party, pv)
  return s

for issue in df.columns[1:]:
  print(print_report(issue))

On the issue of handicapped_infants, support leans Democrat at p=1.28E-18.

On the issue of adoption_of_the_budget_resolution, support leans Democrat at p=2.87E-76.

On the issue of physician_fee_freeze, support leans Republican at p=3.97E-169.

On the issue of el_salvador_aid, support leans Republican at p=1.47E-67.

On the issue of religious_groups_in_schools, support leans Republican at p=1.14E-20.

On the issue of anti_satellite_test_ban, support leans Democrat at p=1.27E-30.

On the issue of aid_to_nicaraguan_contras, support leans Democrat at p=1.49E-53.

On the issue of mx_missile, support leans Democrat at p=4.86E-47.

On the issue of synfuels_corporation_cutback, support leans Democrat at p=2.74E-15.

On the issue of education_spending, support leans Republican at p=2.38E-62.

On the issue of superfund_right_to_sue, support leans Republican at p=3.20E-34.

On the issue of crime, support leans Republican at p=5.10E-46.

On the issue of duty_free_exports, support leans Democrat 

In [167]:
def on_the_issue_of(rs): 
  idx = rs[0]
  pv = rs[1]
  sg = sign(significant_results.loc[first_result[0]][0])
  if (sg==1): 
    party = "Republican"
  else: 
    party = "Democrat"
  sp = "significant result is on the issue of "
  s = "{0}, which came in at {1} in favor of {2}.".format(idx,pv,party)
  return sp + s

print("The most " + on_the_issue_of(first_min))
print("The second-most " + on_the_issue_of(second_min))

The most significant result is on the issue of physician_fee_freeze, which came in at 3.967141133302638e-169 in favor of Republican.
The second-most significant result is on the issue of adoption_of_the_budget_resolution, which came in at 2.872115314395808e-76 in favor of Republican.


In [168]:
significant_results

,T_stat,p_value
handicapped_infants,-9.223178,1.276117e-18
adoption_of_the_budget_resolution,-22.821693,2.872115e-76
physician_fee_freeze,46.101910,3.967141e-169
el_salvador_aid,20.895617,1.465966e-67
religious_groups_in_schools,9.815876,1.142999e-20
anti_satellite_test_ban,-12.448556,1.273630e-30
aid_to_nicaraguan_contras,-17.791848,1.494801e-53
mx_missile,-16.326540,4.863267e-47
synfuels_corporation_cutback,-8.200712,2.743404e-15
education_spending,19.744624,2.383628e-62


In [169]:
agree_idx = significant_results.T_stat.abs().idxmin()
agree_t = significant_results.loc[agree_idx][0]
agree_p = significant_results.loc[agree_idx][1]

agreement = "The t-stat closest to zero is {0}, so the issue of {1} is the closest D and R are to agreeing (at significance {2})".format(agree_t, agree_idx, agree_p)
print(agreement)

The t-stat closest to zero is -4.6756205760407, so the issue of export_administration_act_south_africa is the closest D and R are to agreeing (at significance 3.9212926039422195e-06)
